In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
import os

# Define the classes in your dataset
classes = ['2lanes', '3lanes', 'crossing', 'split4lanes', 'split6lanes', 'transition']

# Load the dataset
data = []
labels = []

# Iterate over the folders/classes
for class_name in classes:
    folder_path = f'H:\\Approach2_sol\\dataset\\train_data\\{class_name}'  # Update the path
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        # Load the image using Pillow (PIL)
        image = Image.open(file_path)
        #image = image.convert('RGB')  # Convert image to RGB mode if needed
        image = image.resize((256, 256))  # Resize the image to your desired dimensions
        
        # Convert the PIL image to a NumPy array
        image_array = np.array(image)
        
        # Append the image to the data list
        data.append(image_array)
        
        # Append the class label to the labels list
        labels.append(class_name)

# Convert the class labels to integer values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, y_encoded, test_size=0.2, random_state=42)

# Define data transforms for data augmentation
data_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = Image.fromarray(self.data[idx])
        
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]

        return image, label

# Create custom datasets and data loaders
train_dataset = CustomDataset(X_train, y_train, transform=data_transform)
test_dataset = CustomDataset(X_test, y_test, transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the ResNet model
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(classes))

# Move the model to the GPU, if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Evaluate the model on the testing set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print("Test Accuracy:", test_accuracy)


c:\Users\mohamad\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mohamad\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100, Loss: 1.4794
Epoch 2/100, Loss: 1.1884
Epoch 3/100, Loss: 1.0921
Epoch 4/100, Loss: 1.0436
Epoch 5/100, Loss: 0.9714
Epoch 6/100, Loss: 0.9084
Epoch 7/100, Loss: 0.8283
Epoch 8/100, Loss: 0.8193
Epoch 9/100, Loss: 0.8300
Epoch 10/100, Loss: 0.7515
Epoch 11/100, Loss: 0.7664
Epoch 12/100, Loss: 0.7349
Epoch 13/100, Loss: 0.7514
Epoch 14/100, Loss: 0.7208
Epoch 15/100, Loss: 0.6813
Epoch 16/100, Loss: 0.6416
Epoch 17/100, Loss: 0.6364
Epoch 18/100, Loss: 0.6236
Epoch 19/100, Loss: 0.6285
Epoch 20/100, Loss: 0.5649
Epoch 21/100, Loss: 0.5790
Epoch 22/100, Loss: 0.5672
Epoch 23/100, Loss: 0.5473
Epoch 24/100, Loss: 0.5128
Epoch 25/100, Loss: 0.4870
Epoch 26/100, Loss: 0.5286
Epoch 27/100, Loss: 0.4961
Epoch 28/100, Loss: 0.5065
Epoch 29/100, Loss: 0.4661
Epoch 30/100, Loss: 0.4363
Epoch 31/100, Loss: 0.4338
Epoch 32/100, Loss: 0.4115
Epoch 33/100, Loss: 0.3869
Epoch 34/100, Loss: 0.3665
Epoch 35/100, Loss: 0.3960
Epoch 36/100, Loss: 0.3959
Epoch 37/100, Loss: 0.4038
Epoch 38/1